In [1]:
# required packages

# nltk.download('wordnet')

# nltk.download('omw-1.4')

# pip install python-docx

In [2]:
# Required import
import nltk
from nltk.corpus import wordnet

import docx2txt
import spacy
import re
from docx import Document

from docx.shared import RGBColor
from docx.oxml import OxmlElement
from docx.oxml.ns import qn

import docx
from docx.enum.text import WD_COLOR_INDEX

import time

In [3]:
start_time = time.time()

In [4]:
# Already existed words in Parties 

Parties = ["Unilateral", "One-Way", "One Way", "company (\"Disclosing Party\")", "company (\"Discloser\")", "corporation (\"Disclosing Party\")", "corporation (\"Discloser\")",  "LLC (\"Disclosing Party\")", "LLC (\"Discloser\")",  "Inc. (\"Disclosing Party\")", "Inc. (\"Discloser\")",  "Incorporated (\"Disclosing Party\")", "Incorporated (\"Discloser\")", "Co. (\"Disclosing Party\")", "Co. (\"Discloser\")"]


In [5]:
# Already existed words in Residuals/ Memories

Residuals = ["residual", "residuals", "memories", "unaided", "memory"]


In [6]:
# Arealdy existed words in Limitation of Liability

Limitation = ["Limitation of Liability", "under no circumstances", "shall be limited", "special, incidental, indirect or consequential damages", "punitive","exemplary","consequential","indirect","incidental"]


In [7]:
# Already existed words in Non-competition

Noncompetition = ["compete", "competition", "non-compete", "non-competition", "non compete", "non competition" ]


In [8]:
# Already existed words in Non-solicitation

Nonsolicitation = ["non-solicitation", "solicit", "non-solicit", "non-servicing",   "nonsolicitation", "nonsolicit", "nonservicing",  "solicit",  "non solicitation", "non solicit", "non servicing",  "no solicit", "no-solicit"]


In [9]:
# Already existed words in Indemnification

Indemnification = ["indemnification", "indemnity", "hold-harmless", "hold harmless", "indemnify", "indemnified", "indemnifying", "defend"]


In [10]:
# Already existed words in Governing Law/ Jurisdiction

Governing = ["Texas", "Italy", "Italian"]

In [11]:
# Already existed words in Exceptions

Exceptions =  ["was communicated by Disclosing Party to an unaffiliated third party free of any obligation of confidence",   "was communicated by Disclosing Party to a third party free of any obligation of confidence", "was communicated by Disclosing Party to  a third party without an obligation of confidence", "was disclosed by Disclosing Party to  a third party without an obligation of confidence", " was disclosed by Discloser to a third party without an obligation of confidentiality"]


In [12]:
# Already existed words for Representatives

# Different from the above, this part is for existence, if exist, yes, if not exist, output a warning sign about "Missing Representatives"

Representatives = ["responsible", "liable"]

In [13]:
# Already existed words for Remedies

# Same as the last one, this part is for existence, if exist, yes, if not exist, output a warning sign about "Missing Remedies"
Remedies = ["injunction", "injunctive","equitable"]


In [14]:
# Already existed words for Privacy / Personal Information
Privacy = ["Personal Data", "Personal Information","PII", "GDPR", "CCPA", "CPRA", "Privacy"]


Test Trial<br>
输入路径

In [15]:
##           ###           ##  EEEEEEEEEEEEEEEE      III
 ##         ## ##         ##   E                     III
  ##       ##   ##       ##    E                     III
   ##     ##     ##     ##     EEEEEEEEEEEEEEE       III
    ##   ##       ##   ##      E                     III
     ## ##         ## ##       E                     III
      ##            ##         EEEEEEEEEEEEEEEE      III

In [16]:
# Test 2
# 改一下你的输入路径 注意r \\ 和 docx别动
doc = docx.Document(r'C:\\Users\\dell-pc\\Desktop\\NDA.docx')



In [17]:
paragraphs = ''

for para in doc.paragraphs:
    paragraphs += para.text

Sentences Segmentation

In [18]:
# Segment paragraph into sentences. 
nlp = spacy.load("en_core_web_sm")

# take unicode string  
sentences = nlp(paragraphs)

sentencess = list(sentences.sents)

separated_sentences = []

# list all sentences
for i in range(len(sentencess)):
    sentence = sentencess[i].text
    
    #sentence_separation = re.split('\t|\n|\n\t', sentence)
    
    sentence = sentence.replace('\n\n', ' ')
    sentences = re.split(r'\.(?=\s|$)', sentence)


    separated_sentences += sentences

final_separated_sentences = [x for x in separated_sentences if x != '']

## final_separated_sentences

In [19]:
# Recall in previous section, we have already find synonyms of dealbreaker words and list them. 

## Parties
## Residuals
## Limitation
## Noncompetition
## Nonsolicitation
## Indemnification
## Representatives
## Remedies
## Privacy
## Governing
## Exceptions


Create dictionary for dealbreaker catergories and words with each category

In [20]:
# create a dictionary of sensitive words, grouped them by different categories. 

dealbreaker_words = {
    "Parties":Parties,
     "Residuals/Memories": Residuals,
     "Limitation of Liability": Limitation,
     "Non-competition": Noncompetition,
     "Non-solicitation":Nonsolicitation,
     "Indemnification": Indemnification,
     "Governing Law/Jurisdiction":Governing,
     "Exceptions":Exceptions,
     "Privacy": Privacy
}

# The reason why not including remedies and representatives is due to the reason that
# their search methods are for existence (recall if they exist, then everything is ok, but if not exist, Warning)

In [21]:
check_list = []
for i in range(len(final_separated_sentences)):
    sentence = final_separated_sentences[i]
    # Iterate through the sensitive words grouped by category
    for category, words in dealbreaker_words.items():
        # Use regular expressions to search for the words in the sentence
        for word in words:
            matches = re.finditer(r'\b'+word+r'\b', sentence, re.IGNORECASE)
            for match in matches:
                check_list.append(match.group())

## check_list

In [22]:
# Clean it for unique words, Upper or lower case matters!

check_list = list(set(check_list))

In [23]:
# Representatives
representative_warning = True

for i in range(len(separated_sentences)):
    sentence = separated_sentences[i]
    for word in Representatives:
        if word in sentence:
            representative_warning = False
            ## print(f"Found '{word}' in sentence: {sentence}")
            break
    else:
        continue

    break


In [24]:
# Remedies
remedies_warning = True

for i in range(len(separated_sentences)):
    sentence = separated_sentences[i]
    for word in Remedies:
        if word in sentence:
            remedies_warning = False
            ## print(f"Found '{word}' in sentence: {sentence}")
            break
    else:
        continue

    break

Export Stage

Trial

In [25]:
##           ###           ##  EEEEEEEEEEEEEEEE      III
 ##         ## ##         ##   E                     III
  ##       ##   ##       ##    E                     III
   ##     ##     ##     ##     EEEEEEEEEEEEEEE       III
    ##   ##       ##   ##      E                     III
     ## ##         ## ##       E                     III
      ##            ##         EEEEEEEEEEEEEEEE      III

In [26]:
# Import document again!
# 改你的路径哦，不要动r \\ 和docx
doc=docx.Document(r'C:\\Users\\dell-pc\\Desktop\\NDA.docx')

In [27]:
def find_key_by_value(value, dictionary):
    keys = []
    for key, val in dictionary.items():
        if isinstance(val, list):
            if any(v.lower() == value.lower() for v in val):
                keys.append(key)
        elif val.lower() == value.lower():
            keys.append(key)
    return keys if keys else None  # if value is not found in any key

Giving comment on dealbreaker words

In [28]:
def split_text(text, word):
    pattern = re.compile(r'([\S\s]*)(\b{})([\S\s]*)'.format(word))
    match = pattern.search(text)
    if match:
        return match.groups()
    return None

def split_Runs(doc,word):
    for p in doc.paragraphs:
        if p.text.find(word) != -1:
            virtualRuns=p.runs
            p.text = ""
            for r in virtualRuns:
                if r.text.find(word) != -1:
                    before, word, after = split_text(r.text, word)
                    p.add_run(before)
                    p.add_run()
                    p.add_run(word)
                    p.add_run(after)
                else:
                    p.add_run(r.text)
    return doc


def style_Token2(doc,word, dictionary1, comment=True):
    for p in doc.paragraphs:
        for i,r in enumerate(p.runs):
            if p.runs[i].text.find(word) != -1:
                p.runs[i].font.highlight_color = WD_COLOR_INDEX.RED
                if comment:
                    corresponded_key = find_key_by_value(word,dictionary1)
                    p.runs[i-1].add_comment(f'Wanring: \"{word}\" is classified as a category {corresponded_key} dealbreaker word.\nStrongly Advice its Immediate Removal',author="DG1")


    return doc


# check_list

for keyword in check_list:
    doc=split_Runs(doc,keyword)    

for keyword in check_list:
    doc=style_Token2(doc,keyword, dealbreaker_words,True)

Giving Comment on Representative and Remedies <br>
if they exist, nothing will happen <br>
if they do not exist, output a warning comment

In [29]:
def check_existence_for_remedy_representative(doc,warning_1 = True, warning_2 = True):
    if warning_1:
        paragraph1 = doc.add_paragraph()
        comment1 = paragraph1.add_comment("Warning: This document is lack of Representative Category information", author = 'DG1')
    if warning_2:
        paragraph2 = doc.add_paragraph()
        comment2 = paragraph2.add_comment("Warning: This document is lack of Remedy Category information",author = 'DG1')


    return doc

doc = check_existence_for_remedy_representative(doc,representative_warning,remedies_warning)

Output Path<br>
输出路径

In [30]:
##           ###           ##  EEEEEEEEEEEEEEEE      III
 ##         ## ##         ##   E                     III
  ##       ##   ##       ##    E                     III
   ##     ##     ##     ##     EEEEEEEEEEEEEEE       III
    ##   ##       ##   ##      E                     III
     ## ##         ## ##       E                     III
      ##            ##         EEEEEEEEEEEEEEEE      III

In [31]:
# 改你的输出路径， 不要动r, \\, docx
doc.save(r'C:\\Users\\dell-pc\\Desktop\\Only_NDA2.docx')


In [32]:
end_time = time.time()

In [33]:
total_time = end_time - start_time

In [34]:
print(f"Total time taken: {total_time:.2f} seconds")

Total time taken: 5.43 seconds
